In [1]:
import os
import sys
import json
import albumentations as A

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from building_footprint_segmentation.segmentation import init_segmentation, read_trainer_config
from building_footprint_segmentation.helpers.callbacks import CallbackList, load_callback
from building_footprint_segmentation.trainer import Trainer

### Initalize type of segmentation to run for building footprint

In [2]:
segmentation = init_segmentation("binary")

### Defining augmenters to use

In [3]:
augmenters = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    
    # 추가
    A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=0.5, border_mode=0),
    
    A.PadIfNeeded(min_height=128, min_width=128, always_apply=True, border_mode=0),
    A.RandomCrop(height=128, width=128, always_apply=True),

    A.OneOf(
    [
        A.CLAHE(p=1),
        A.RandomBrightness(p=1),
        A.RandomGamma(p=1),
    ],
    p=0.8,
    ),
    
    A.OneOf(
        [
            A.Blur(blur_limit=3, p=1),
            A.MotionBlur(blur_limit=3, p=1),
        ],
        p=0.8,
    ),
    
    A.OneOf(
        [
            A.RandomContrast(p=1),
            A.HueSaturationValue(p=1),
        ],
        p=0.8,
    ),
    
])

/home/complexion/miniconda3/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/complexion/miniconda3/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


### Load essential segmentation objects for training

In [4]:
model = segmentation.load_model(name="ReFineNet")
criterion = segmentation.load_criterion(name="Dice",dice_weight=0.7)


/home/complexion/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/complexion/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
loader = segmentation.load_loader(
    root_folder=r"data",
    image_normalization="divide_by_255",
    label_normalization="binary_label",
    augmenters=augmenters,
    batch_size=16,
)


In [6]:
metrics = segmentation.load_metrics(
    data_metrics=["precision", "f1", "recall", "iou", "dice"]
)

# optimizer = segmentation.load_optimizer(model, name="Adam")
optimizer = segmentation.load_optimizer(model, name="SGD",lr=0.05)

callbacks = CallbackList()
# Ouptut from all the callbacks caller will be stored at the path specified in log_dir
for caller in  ["TrainChkCallback", "TimeCallback", "TensorBoardCallback", "TrainStateCallback"]:
    callbacks.append(load_callback(r"out_data", caller))

### Initalize the Trainer object


In [7]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    loader=loader,
    metrics=metrics,
    callbacks=callbacks,
    optimizer=optimizer,
    scheduler=None,
)

### Start training


In [8]:
trainer.train(start_epoch=0, end_epoch=70)

Epoch 0, lr 0.05: 100%|██████████| 74576/74576 [11:57<00:00, 103.97it/s, loss=0.31134]


>>Py-OneLiner : Loss : train: 0.31134341575205327, validation: 0.2385651854920809
>>Py-OneLiner : Train Metric : precision : 0.62394, f1 : 0.58532, recall : 0.59714, iou : 0.43467, dice : 0.58532
>>Py-OneLiner : Valid Metric : precision : 0.72653, f1 : 0.72928, recall : 0.74463, iou : 0.58014, dice : 0.72928

Epoch 1, lr 0.05: 100%|██████████| 74576/74576 [11:50<00:00, 104.91it/s, loss=0.29168]


>>Py-OneLiner : Loss : train: 0.2916848935186863, validation: 0.2183259292623075m
>>Py-OneLiner : Train Metric : precision : 0.70225, f1 : 0.66825, recall : 0.67064, iou : 0.51723, dice : 0.66825
>>Py-OneLiner : Valid Metric : precision : 0.76746, f1 : 0.74989, recall : 0.74273, iou : 0.60561, dice : 0.74989

Epoch 2, lr 0.05: 100%|██████████| 74576/74576 [11:49<00:00, 105.10it/s, loss=0.24746]


...

>>Py-OneLiner : Loss : train: 0.17628315549343823, validation: 0.14355785890380626
>>Py-OneLiner : Train Metric : precision : 0.81654, f1 : 0.79928, recall : 0.79774, iou : 0.67751, dice : 0.79928
>>Py-OneLiner : Valid Metric : precision : 0.83150, f1 : 0.83681, recall : 0.84839, iou : 0.72422, dice : 0.83681

Epoch 68, lr 0.05: 100%|██████████| 74576/74576 [11:50<00:00, 104.97it/s, loss=0.18270]


>>Py-OneLiner : Loss : train: 0.18270425166934728, validation: 0.14426621451585211
>>Py-OneLiner : Train Metric : precision : 0.81473, f1 : 0.79877, recall : 0.79830, iou : 0.67713, dice : 0.79877
>>Py-OneLiner : Valid Metric : precision : 0.83267, f1 : 0.83579, recall : 0.84560, iou : 0.72306, dice : 0.83579

Epoch 69, lr 0.05: 100%|██████████| 74576/74576 [11:50<00:00, 104.94it/s, loss=0.16042]


>>Py-OneLiner : Loss : train: 0.16041698902845383, validation: 0.14016552296263643
>>Py-OneLiner : Train Metric : precision : 0.81556, f1 : 0.80054, recall : 0.80139, iou : 0.67927, dice : 0.80054
>>Py-OneLiner : Valid Metric : precision : 0.83278, f1 : 0.84048, recall : 0.85453, iou : 0.72985, dice : 0.84048
>>Py-OneLiner : Training Complete
>>Py-OneLiner : Run Time : 0:Days 17:Hours 17:Minutes 49:Seconds

### Resuming Training


70번까진 batch 8, 120번까진 batch 16

In [8]:
path_to_saved_state = r"out_data/20230727-010621/state/default.pt"

# if new_end_epoch is None, the trainer will resume and run till end_epoch, the value of which is saved in state file 
trainer.resume(state=path_to_saved_state, new_end_epoch=120)

Epoch 118, lr 0.05: 100%|██████████| 74576/74576 [10:25<00:00, 119.30it/s, loss=0.16353]

>>Py-OneLiner : Validation : 1/1428 ETA -- 0:Days 0:Hours 1:Minutes 4:Seconds

>>Py-OneLiner : Loss : train: 0.1635326200723648, validation: 0.1312586806730187m
>>Py-OneLiner : Train Metric : precision : 0.84574, f1 : 0.83672, recall : 0.83365, iou : 0.72418, dice : 0.83672
>>Py-OneLiner : Valid Metric : precision : 0.84497, f1 : 0.85093, recall : 0.86019, iou : 0.74306, dice : 0.85093

Epoch 119, lr 0.05: 100%|██████████| 74576/74576 [09:55<00:00, 125.18it/s, loss=0.13708]

>>Py-OneLiner : Validation : 1/1428 ETA -- 0:Days 0:Hours 0:Minutes 55:Seconds

>>Py-OneLiner : Loss : train: 0.13707527238875628, validation: 0.12488776505650843
>>Py-OneLiner : Train Metric : precision : 0.84522, f1 : 0.83601, recall : 0.83341, iou : 0.72343, dice : 0.83601
>>Py-OneLiner : Valid Metric : precision : 0.85966, f1 : 0.85768, recall : 0.85872, iou : 0.75348, dice : 0.85768
>>Py-OneLiner : Training Complete
>>Py-OneLiner : Run Time : 0:Days 0:Hours 26:Minutes 32:Seconds